<h3>Image Stitching</h3>

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt 

In [2]:
IMG_PATH = '../images/'

img1 = cv2.imread(IMG_PATH + 'palace_1.jpg')
img2 = cv2.imread(IMG_PATH + 'palace_2.jpg')
img3 = cv2.imread(IMG_PATH + 'palace_3.jpg')
img4 = cv2.imread(IMG_PATH + 'palace_4.jpg')


(177, 317, 3)
(177, 318, 3)
(178, 317, 3)
(178, 318, 3)


In [11]:
import cv2
import numpy as np

# Function to find matches between two images
def find_matches(image1, image2):
    sift = cv2.SIFT_create()

    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    return keypoints1, keypoints2, good_matches

# Function to create a panorama from two images
def create_panorama(image1, image2):
    keypoints1, keypoints2, matches = find_matches(image1, image2)

    if len(matches) > 4:
        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        height, width = image1.shape[:2]
        warped_image = cv2.warpPerspective(image2, M, (width + image2.shape[1], height))
        warped_image[:, :width] = image1

        return warped_image

    else:
        print("Not enough matches to create a panorama between these two images.")
        return None
 
# Create panoramas between images
panorama_12 = create_panorama(img1, img2)
panorama_34 = create_panorama(img3, img4)

# Combine the panoramas to create the final panorama
final_panorama = create_panorama(panorama_12, panorama_34)

# Display the final panorama
cv2.imshow('Final Panorama', final_panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()


: 